## TRAIN FC

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import seed
seed(12345)

The following is to disable **GPU**. In my notebook, I cannot use more than one item per mini-batch with my GPU.

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten

#from tensorflow_addons.losses import SigmoidFocalCrossEntropy  # tfa.losses.SigmoidFocalCrossEntropy()

In [ ]:
#  Checkpoints capture the exact value of all parameters (tf.Variable objects)
#  used by a model. Checkpoints do not contain any description of the computation
#  defined by the model and thus are typically only useful when source code 
#  that will use the saved parameter values is available.
from keras.callbacks import ModelCheckpoint

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
tf.config.get_visible_devices()

Model:

In [ ]:
import sys # to load custom python code
sys.path.append('...\\Source')
from unet import unet  # Model

Loss:

In [ ]:
from loss import *  # Custom losses

In [ ]:
from patchProcessAssemble import Normalize
# def Normalize(x):
#     # see tf.image.per_image_standardization
#     N      = len(x)
#     mean   = np.mean(x)
#     stddev = np.std(x)
#     stddev = np.maximum(stddev, 1.0/np.sqrt(N)) # protect againts division by zero
#     return (x - mean) / stddev  

Data generator:

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for keras'
    def __init__(self,dpath,fpath,data_IDs, batch_size=1, dim=(128,128,128), 
                 n_channels=1, shuffle=True):
        'Initialization'
        self.dim   = dim
        self.dpath = dpath
        self.fpath = fpath
        self.batch_size = batch_size
        self.data_IDs   = data_IDs
        self.n_channels = n_channels
        self.shuffle    = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data_IDs)/self.batch_size))

    def __getitem__(self, index):
        'Generates one batch of data'
        # Generate indexes of the batch
        bsize = self.batch_size
        indexes = self.indexes[index*bsize:(index+1)*bsize]

        # Find list of IDs
        data_IDs_temp = [self.data_IDs[k] for k in indexes]

        # Generate data
        X, Y = self.__data_generation(data_IDs_temp)

        return X, Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data_IDs))
        if self.shuffle == True:
              np.random.shuffle(self.indexes)

    def __data_generation(self, data_IDs_temp):
        'Generates data containing batch_size samples'
        # Initialization
        # JLG: data stream is read:
        gx  = np.fromfile(self.dpath+str(data_IDs_temp[0])+'.dat',dtype=np.single)
        fx  = np.fromfile(self.fpath+str(data_IDs_temp[0])+'.dat',dtype=np.single)
        # JLG: data is shaped into a cube
        gx = np.reshape(gx,self.dim)
        fx = np.reshape(fx,self.dim)
        # JLG: data is whitened (normalized):
        gx = Normalize(gx)
        # JLG: data is transposed to conform with (time,x,y); meaning that the original input is (x,y,time).
        gx = np.transpose(gx)
        fx = np.transpose(fx)
        #in seismic processing, the dimensions of a seismic array is often arranged as
        #a[n3][n2][n1] where n1 represnts the vertical dimenstion. This is why we need 
        #to transpose the array here in python 
        
        # Generate data
        
        # Local CPU (too slow: use to run for a few epochs for testing. I can use more than one item per mini-batch)
        X = np.zeros((2, *self.dim, self.n_channels),dtype=np.single) # JLG: e.g., X is (2,128x128x128,1)
        Y = np.zeros((2, *self.dim, self.n_channels),dtype=np.single)
        X[0,] = np.reshape(gx, (*self.dim,self.n_channels))  # JLG: the original `image`
        Y[0,] = np.reshape(fx, (*self.dim,self.n_channels))  
        X[1,] = np.reshape(np.flipud(gx), (*self.dim,self.n_channels)) # JLG: upside down
        Y[1,] = np.reshape(np.flipud(fx), (*self.dim,self.n_channels))  
        
        '''
        # Local GPU (use one item per mini-batch due to limited memory resources)
        X = np.zeros((1, *self.dim, self.n_channels),dtype=np.single) 
        Y = np.zeros((1, *self.dim, self.n_channels),dtype=np.single)
        X[0,] = np.reshape(gx, (*self.dim,self.n_channels))  
        Y[0,] = np.reshape(fx, (*self.dim,self.n_channels))  
        
        '''
        '''     
        #Google Colaboratory GPU or local CPU (for a few epochs):
        X = np.zeros((4, *self.dim, self.n_channels),dtype=np.single) 
        Y = np.zeros((4, *self.dim, self.n_channels),dtype=np.single)        
        for i in range(4):
            X[i,] = np.reshape(np.rot90(gx,i,(2,1)), (*self.dim,self.n_channels))
            Y[i,] = np.reshape(np.rot90(fx,i,(2,1)), (*self.dim,self.n_channels))  
        '''
        return X,Y

Training function:

In [ ]:
def goTrain():
    
    # input image dimensions
    params = {'batch_size':1,
            'dim':(128,128,128),
            'n_channels':1,
            'shuffle': True}
    
    seismPathT = "D:\\Tesis\\faultSeg-master\\data\\train\\seis\\"
    faultPathT = "D:\\Tesis\\faultSeg-master\\data\\train\\fault\\"

    seismPathV = "D:\\Tesis\\faultSeg-master\\data\\validation\seis\\"
    faultPathV = "D:\\Tesis\\faultSeg-master\\data\\validation\\fault\\"
    
    # checkpoint
    filepath="D:\\Tesis\\Notebooks_JLG\\checkpoint-{epoch:02d}.hdf5"
    checkpoint = ModelCheckpoint(filepath, 
                                 monitor=tf.keras.metrics.RootMeanSquaredError(), #'val_acc', 
                                 verbose=1, 
                                 save_best_only=False,
                                 mode='max')
    
    # ORIGINAL
    EPOCHS  = 20
    N_train = 200
    N_valid = 20
    
    # FAST TEST
    #EPOCHS  = 10
    #N_train = 1
    #N_valid = 1
    
    
    # Reducir el número para entrenar con un número menor de ejemplos
    train_ID = range(N_train)
    valid_ID = range(N_valid)
    
    
    train_generator = DataGenerator(dpath=seismPathT,fpath=faultPathT,
                                  data_IDs=train_ID,**params)
    
    valid_generator = DataGenerator(dpath=seismPathV,fpath=faultPathV,
                                  data_IDs=valid_ID,**params)
    
    model = unet(input_size=(None, None, None,1))
    
#     # Así está en el original:
#     model.compile(optimizer=Adam(learning_rate=1e-4), 
#                  loss='binary_crossentropy', 
#                  metrics=[tf.keras.metrics.RootMeanSquaredError(),
#                          tf.keras.metrics.Accuracy()])

    # PARA ANALIZAR: DICE LOSS
    model.compile(optimizer=Adam(learning_rate=1e-4), 
                 loss=smooth_dice_lossGPT, 
                 metrics=[tf.keras.metrics.RootMeanSquaredError(),
                         tf.keras.metrics.BinaryAccuracy(),
                         tf.keras.metrics.TrueNegatives(),
                          tf.keras.metrics.TrueNegatives(),
                          tf.keras.metrics.TruePositives(),
                          tf.keras.metrics.FalsePositives(),
                          tf.keras.metrics.FalseNegatives(),])
    
#     model.compile(optimizer=Adam(learning_rate=1e-4), 
#                  loss=SigmoidFocalCrossEntropy(from_logits = False), # unet calcula sigmoid en last layer
#                  metrics=[tf.keras.metrics.RootMeanSquaredError(),
#                          tf.keras.metrics.Accuracy()])
    
#     # Loss escrito para entender:
   # model.compile(optimizer = Adam(learning_rate=1e-4), 
    #               loss      = cross_entropy_balanced_JLG,
     #              metrics   = [tf.keras.metrics.RootMeanSquaredError(),                               
      #                          tf.keras.metrics.Accuracy()])

    
     # Utilizo el balanced loss propuesto por el autor:
#     model.compile(optimizer=Adam(lr=1e-4), 
#                   loss = cross_entropy_balanced, # loss='binary_crossentropy'
#                   metrics=['accuracy'])

# Utilizo el loss propuesto por el autor, pero escrito por mi:
# AQUI TIENE SENTIDO QUE ACCURACY == 0 Y QUE HAYA QUE MEDIR CON RMSE
#     model.compile(optimizer = Adam(learning_rate=1e-4), 
#                   loss      = cross_entropy_balanced_JLG,
#                   metrics   = [tf.keras.metrics.RootMeanSquaredError(),                               
#                                tf.keras.metrics.Accuracy(),
#                                tf.keras.metrics.Precision(),
#                                tf.keras.metrics.Recall()])

# Utilizo el balanced loss propuesto por el autor, pero escrito por mi (con binary cross-entropy):
#     model.compile(optimizer=Adam(learning_rate=1e-4), 
#                   loss = BinaryCrossentropyBalanced, # loss='binary_crossentropy'
#                   metrics=['accuracy'])
 
    
    print("Data is prepared. Ready to train !") # JLG
    
    # Fit the model
    history = model.fit(x = train_generator,
                        validation_data=valid_generator,
                        epochs=EPOCHS,  # few epochs to check all works
                        callbacks=[checkpoint],
                        verbose=1)    
    
    #model.save('trained_model/fsegmentation_JLG.hdf5') # the whole model and history  
    model.save_weights('D:\\Tesis\\Notebooks_JLG\\weights.h5') # just the neccesary.  
    return model, history

### Preliminaries

In [ ]:
# Check model architecture:
model_ = unet(input_size=(None, None, None,1))
model_.summary()

In [ ]:
# Check model's IO: "that the model outputs something"

# training image dimensions
n1, n2, n3 = 128, 128, 128

gx  = np.random.normal(size=n1*n2*n3).reshape(n1,n2,n3)
# convertir a tensor para el modelo:
x   = np.reshape(gx,(1,n1,n2,n3,1))
# predecir
Y   = model_.predict(x,verbose=1)
# Remuevo dimensiones para graficar
x = x.squeeze() # (1,n1,n2,n3,1) => (n1,n2,n3)
Y = Y.squeeze() # (1,n1,n2,n3,1) => (n1,n2,n3)
# Plot
plt.figure()
plt.subplot(1,2,1)
plt.title("Input (slice)")
plt.imshow(x[:,:,n3//2])
plt.subplot(1,2,2)
plt.title("Output (slice)")
plt.imshow(Y[:,:,n3//2])
plt.show()

# Do the training

I use a few epochs to see if all works. Also, to check processing times.

In [ ]:
model, history = goTrain()
print("Done.")

In [ ]:
# Visualize accuracy and loss over epochs:
#showHistory(history)
np.save('...\\history.npy',history.history)


### HISTORY

In [ ]:
history=np.load('...\\history.npy', allow_pickle=True).item() #cargar desde el directorio donde esté history

In [ ]:
history.keys()

In [ ]:
def showHistory(history):
    # list all data in history
    #print(history)
    
    fig = plt.figure(figsize=(10,6))    
    # summarize history for rmse
    plt.plot(history['root_mean_squared_error'],linestyle='-', marker='o', color="black") # JLG
    plt.plot(history['val_root_mean_squared_error'],linestyle='dashed', marker=(5,1), color="tomato") # JLG
    #plt.title('Error Medio Cuadrático del modelo',fontsize=20)
    plt.ylabel('EMC',fontsize=20)
    plt.xlabel('Época',fontsize=20)
    #plt.legend(['Entrenamiento', 'Testeo'], loc='upper right',fontsize=18)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.tick_params(axis='both', which='minor', labelsize=18)
    epochs = range(1, len(history['binary_accuracy'])+1) # assuming history is a dictionary containing the training and validation accuracy for each epoch
    tick_positions = np.arange(0, len(epochs))
    xtick_labels = [str(x) if i%2==0 else "" for i,x in enumerate(tick_positions)]
    plt.xticks(tick_positions, xtick_labels)
    fig.savefig("D:\\Tesis\\Notebooks_JLG\\checkpoints\\history_EMC.jpg",bbox_inches='tight')
    plt.show()
   

    # summarize history for loss
    fig = plt.figure(figsize=(10,6))
    plt.plot(history['loss'],linestyle='-', marker='o',color="black")
    plt.plot(history['val_loss'],linestyle='dashed', marker=(5,1), color="tomato")
    #plt.title('Pérdida del modelo',fontsize=20)
    plt.ylabel('Pérdida',fontsize=20)
    plt.xlabel('Época',fontsize=20)
    xtick_positions=np.arange(0,20)
    xtick_labels = [str(x) if i%2==0 else "" for i,x in enumerate(xtick_positions)]
    plt.xticks(xtick_positions, xtick_labels)
    #plt.legend(['Entrenamiento', 'Testeo'], loc='upper right',fontsize=18)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.tick_params(axis='both', which='minor', labelsize=18)
    fig.savefig("D:\\Tesis\\Notebooks_JLG\\checkpoints\\history_Loss.jpg", bbox_inches='tight')
    plt.show()
    
    # summarize history for accuracy
    fig= plt.figure(figsize=(10,6))
    plt.plot(history['binary_accuracy'],linestyle='-', marker='o', color="black") # JLG
    plt.plot(history['val_binary_accuracy'],linestyle='dashed', marker=(5,1), color="tomato") # JLG
    #plt.title('Precisión del Modelo',fontsize=20)
    plt.ylabel('Precisión',fontsize=20)
    tick_positions = np.arange(0, 1.5, 0.1)
    plt.yticks(tick_positions)
    plt.ylim(0, 1.0)
    xtick_positions2=np.arange(0,20)
    xtick_labels = [str(x) if i%2==0 else "" for i,x in enumerate(xtick_positions)]
    plt.xticks(xtick_positions,xtick_labels)
    plt.xlabel('Época',fontsize=20)
    #plt.legend(['Entrenamiento', 'Testeo'], loc='lower right',fontsize=15)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.tick_params(axis='both', which='minor', labelsize=18)
    fig.savefig("D:\\Tesis\\Notebooks_JLG\\checkpoints\\history_BA.jpg", bbox_inches='tight')
    plt.show()
    
  
    return None


#lw=2

In [ ]:
showHistory(history)

In [ ]:
  # Compute the ROC curve
tp=history['true_positives']
tp=np.array(tp)
tn=history['true_negatives']
tn=np.array(tn)
fp=history['false_positives']
fp=tp=np.array(fp)
fn=history['false_negatives']
fn=tp=np.array(fn)
fpr=fp / (fp + tn)
tnr=tn / (tn + fp)
pr=history['binary_accuracy']
    #auc = np.trapz(tpr, x=fpr)
print("AUC:", fpr, tnr)
print("TP:",tp)
print("TN:",tn)
print("FP:",fp)
print("FN:",fn)
print("Precisión", pr)




# Plot the ROC curve
    #fig=plt.scatter(fpr, tnr)
    #plt.xlabel('False Positive Rate')
    #plt.ylabel('True Positive Rate')
    #plt.show()
    
    #Plot the Recall-Precision curve
recall = tp/(tp+fn)
precision=tp/(tp+fp)
print("AUC:", recall, precision)
    #fig2=plt.scatter(recall,precision)
    #plt.xlabel('Recall')
    #plt.ylabel('Precision')
F1=(2 * precision * recall) / (precision + recall)
print("F1: ", F1)
    
    

# EXTRA METRICAS

In [ ]:
from sklearn.metrics import confusion_matrix

#import seaborn as sn
import pandas as pd
from tensorflow.keras.models import load_model # to load model

In [ ]:
n1,n2,n3 = 128,128,128
model = unet(input_size=(n1, n2, n3, 1))        # set architecture
#model = Bigunet(input_size=(n1, n2, n3, 1))        # set architecture

#model.load_weights("D:\\Tesis\model\\pretrained_model.hdf5") # get weights from original model from git-hub
model.load_weights("...\\weights.h5")# get weights from localGPU

#model.load_weights("D:\\Tesis\\Notebooks_JLG\\Entrenamiento_Collab\\fseg_colab_dice_loss_weights.h5")# get weights from Colab (trained on GPU, smothed dice loss function)


#model.summary()                                   # check summary

# Check prediction

n1,n2,n3 = 128,128,128
gx = np.fromfile("D:\\Tesis\\faultSeg-master\\data\\train\\seis\\9.dat",dtype=np.single)#.squeeze()
fx = np.fromfile("D:\\Tesis\\faultSeg-master\\data\\train\\fault\\9.dat",dtype=np.single)#.squeeze()
fx = tf.reshape(fx,(1,n1,n2,n3,1))

# WATCH OUT FOR NORMALIZATION AND TRANPOSITION:
gx  = Normalize(gx) # check what happens if we do not normalize the input volume ...
gx  = np.reshape(gx,(n1,n2,n3))
#gx  = tf.image.per_image_standardization(gx) # otra forma de normalizar (equivalente a Normalize() )
gx  = tf.transpose(gx)
gx  = tf.reshape(gx,(1,n1,n2,n3,1))
Y   = model.predict(gx,verbose=1)
Y   = tf.transpose(Y)
gx  = tf.transpose(gx)



gx = tf.squeeze(gx).numpy()
Y  = tf.squeeze(Y).numpy()
fx = tf.squeeze(fx).numpy()

threshold = 0.5 # PLAY WITH THIS
Y[ Y <= threshold ] = 0.  
Y[ Y > threshold  ] = 1.

#tf.shape(Y[:,:,50])



In [ ]:
#extra metricas para un cubo en particular
matrix0=np.zeros((2,2))


for i in range(128):
    for j in range(128):
        matrix = confusion_matrix(fx[:,i,j], Y[:,i,j])
        matrix0 +=matrix

print(matrix0)




tn, fp, fn, tp = matrix0.ravel()



fpr=fp / (fp + tn)
print("False positive Rate: ", fpr)


tnr=tn / (tn + fp)
print("True Negative Rate: ", tnr)
    


specificity = tn/(tn+fp)
print("Specificity: ", specificity)

#For imbalanced learning, recall is typically used to measure the coverage of the minority class. 
#— Page 27, Imbalanced Learning: Foundations, Algorithms, and Applications, 2013.

recall = tp/(tp+fn)
print("Recall o Sensitivity: ", recall)


precision=tp/(tp+fp)
print("Precision: ", precision)

#In imbalanced datasets, the goal is to improve recall without hurting precision. These goals, however, are often conflicting, since in order to increase the TP for the minority class, the number of FP is also often increased, resulting in reduced precision.
#— Page 55, Imbalanced Learning: Foundations, Algorithms, and Applications, 2013.
#F-Measure provides a way to combine both precision and recall into a single measure that captures both properties.

F1=(2 * precision * recall) / (precision + recall)
print("F1: ", F1)

# Accuracy
m = tf.keras.metrics.Accuracy()
m.update_state(Y, fx)
print("Accuracy:",m.result().numpy())





### Evaluate trained model

In [ ]:
k1,k2,k3 = 50,110,60
plt.figure(figsize=(10,10))
#inline slice
plt.subplot(1, 3, 1)
plt.title("Input")
plt.imshow(np.transpose(gx[k1,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.axis("off")
plt.subplot(1, 3, 2)
plt.title("True")
plt.imshow(np.transpose(fx[k1,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.axis("off")
plt.subplot(1, 3, 3)
plt.title("Predicted")
plt.imshow(np.transpose(Y[k1,:,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.axis("off")
# xline slice
plt.figure(figsize=(10,10))
plt.subplot(1, 3, 1)
plt.imshow(np.transpose(gx[:,k2,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.axis("off")
plt.subplot(1, 3, 2)
plt.imshow(np.transpose(fx[:,k2,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.axis("off")
plt.subplot(1, 3, 3)
plt.imshow(np.transpose(Y[:,k2,:]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.axis("off")
plt.show()
# time-slice
plt.figure(figsize=(10,10))
plt.subplot(1, 3, 1)
plt.imshow(np.transpose(gx[...,k3]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.axis("off")
plt.subplot(1, 3, 2)
plt.imshow(np.transpose(fx[...,k3]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.axis("off")
plt.subplot(1, 3, 3)
plt.imshow(np.transpose(Y[...,k3]),cmap=plt.cm.bone,interpolation='nearest',aspect=1)
plt.axis("off")
plt.show()

In [ ]:
# Get validation data
params = {'batch_size':1,'dim':(128,128,128),'n_channels':1,'shuffle': True}

seismPathV = "D:\\Tesis\\faultSeg-master\\data\\validation\seis\\"
faultPathV = "D:\\Tesis\\faultSeg-master\\data\\validation\\fault\\"

N_valid    = 10 # How many pairs to evaluate
valid_ID   = range(N_valid)
   

validation_data = DataGenerator(dpath=seismPathV,fpath=faultPathV,
                                data_IDs=valid_ID,**params)    

Evaluate trained model in `N_valid` validation data pairs:

In [ ]:
# Evaluate all data pairs in validation data:
#model.evaluate(validation_data)
n1,n2,n3 = 128,128,128
model = unet(input_size=(n1, n2, n3, 1))
model.load_weights("D:\\Tesis\\Notebooks_JLG\\weights.h5") # get weights from localGPU
model.compile(optimizer = Adam(learning_rate=1e-4), 
                   loss      = smooth_dice_lossGPT, 
                   metrics   = [tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.BinaryAccuracy()])
model.evaluate(validation_data)

Evaluate trained model in one mini-batch of validation data:

In [ ]:
# Evaluate just one mini-batch
X,Y = validation_data.__getitem__(0)
print(tf.shape(X))
print(tf.shape(Y))
model.evaluate(X,Y)

Load a **checkpoint** and evaluate:

In [ ]:
# Load a checkpoint and evaluate
model_chkp = unet(input_size=(None, None, None, 1))
model_chkp.load_weights("...\\checkpoints\\fsegmentation-14.hdf5")
model_chkp.compile(optimizer = Adam(learning_rate=1e-4), 
                   loss      = smooth_dice_lossGPT, 
                   metrics   = [tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.BinaryAccuracy()])
    
model_chkp.evaluate(validation_data)